### 0.0 import libraries

In [1]:
import os

import pyvista as pv
import trimesh as tm
import numpy as np
import topogenesis as tg
import pickle as pk

### 0.1 import environment

In [2]:
# base 'rubik's cube'
unit = 75
division = 5
macrovoxel = int(unit/division)
base = tm.creation.box((unit,unit,unit))
newshape = base.apply_translation([60, -60, 0]) # cube is moved to correct position

In [8]:
# load context file. Currently set as Rotterdam area from spatial computing course
context_path = os.path.relpath('../../input/immediate_context.obj')
context = tm.load(context_path)
context = context.apply_translation([0, 0, -unit/2]) # Environment is moved down to accomodate accurate voxelization TODO: change to defaults, remember for simulations!

# save obj using pickle TODO: pickle
export_path = os.path.relpath('../../data/movedcontext.obj')
contextnew = context.copy()
# contextnew.export(export_path)
# context.export(export_path)

### 0.2 create macro voxels

In [4]:
# voxelize the cube
base_lattice = tg.lattice(newshape.bounds, unit=[macrovoxel,macrovoxel,macrovoxel], default_value=1, dtype=int)

# take inside values only
interior = newshape.contains(base_lattice.centroids) 
interior_array = interior.reshape(base_lattice.shape)

# convert the interior array into a lattice
envelope_lattice = tg.to_lattice(interior_array, base_lattice.minbound, base_lattice.unit)

### 0.3 saving the lattice

In [5]:
save_path = os.path.relpath('../../data/macrovoxels.csv')
envelope_lattice.to_csv(save_path)

extents_path = os.path.relpath("../../data/maximumextents.obj")
pk.dump(newshape, open(extents_path, "wb"))

### 0.4 visualization

In [6]:
# convert to trimesh definition, taken from spatial computing workshop
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# visualize the mesh using pyvista plotter

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding context, base shape, and lattice
p.add_mesh(tri_to_pv(contextnew), opacity=0.5, color='lightgray')
#p.add_mesh(tri_to_pv(newshape), color='#abd8ff', opacity=0.9)
base_lattice.fast_vis(p)

p.show(use_ipyvtk=True, screenshot='lattice.png')


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(788.6785661399646, 711.2621611399645, 723.5957336399646),
 (65.08283250000001, -12.333572500000002, 0.0),
 (0.0, 0.0, 1.0)]